# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 27 2023 10:58AM,256079,19,OPUS-104420,Opus Life Sciences,Released
1,Jan 27 2023 10:57AM,256078,19,MSP217972,"Methapharm, Inc.",Released
2,Jan 27 2023 10:57AM,256078,19,MSP217973,"Methapharm, Inc.",Released
3,Jan 27 2023 10:57AM,256078,19,MSP217940,Methapharm-G,Released
4,Jan 27 2023 10:51AM,256076,21,PO#1453367/1453370 Samples,"NBTY Global, Inc.",Released
5,Jan 27 2023 10:31AM,256074,10,PRF-46494-BO,Bio-PRF,Released
6,Jan 27 2023 10:31AM,256074,10,PRF-46863-BO,Bio-PRF,Released
7,Jan 27 2023 10:31AM,256074,10,PRF-47521,Bio-PRF,Released
8,Jan 27 2023 10:31AM,256074,10,PRF-47738-BO,Bio-PRF,Released
9,Jan 27 2023 10:31AM,256074,10,PRF-47750-BO,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,256073,Released,5
18,256074,Released,20
19,256076,Released,1
20,256078,Released,3
21,256079,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256073,NaN,NaN,5.0
256074,NaN,NaN,20.0
256076,NaN,NaN,1.0
256078,NaN,NaN,3.0
256079,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256028,0.0,0.0,1.0
256030,0.0,0.0,1.0
256038,0.0,0.0,1.0
256046,3.0,31.0,8.0
256050,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256028,0,0,1
256030,0,0,1
256038,0,0,1
256046,3,31,8
256050,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256028,0,0,1
1,256030,0,0,1
2,256038,0,0,1
3,256046,3,31,8
4,256050,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256028,,,1
1,256030,,,1
2,256038,,,1
3,256046,3,31,8
4,256050,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 27 2023 10:58AM,256079,19,Opus Life Sciences
1,Jan 27 2023 10:57AM,256078,19,"Methapharm, Inc."
3,Jan 27 2023 10:57AM,256078,19,Methapharm-G
4,Jan 27 2023 10:51AM,256076,21,"NBTY Global, Inc."
5,Jan 27 2023 10:31AM,256074,10,Bio-PRF
25,Jan 27 2023 10:29AM,256073,10,Eywa Pharma Inc.
30,Jan 27 2023 9:56AM,256070,10,ISDIN Corporation
31,Jan 27 2023 9:48AM,256067,10,"Senseonics, Incorporated"
32,Jan 27 2023 9:44AM,256064,10,Bio-PRF
33,Jan 27 2023 9:40AM,256063,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 27 2023 10:58AM,256079,19,Opus Life Sciences,,,1
1,Jan 27 2023 10:57AM,256078,19,"Methapharm, Inc.",,,3
2,Jan 27 2023 10:57AM,256078,19,Methapharm-G,,,3
3,Jan 27 2023 10:51AM,256076,21,"NBTY Global, Inc.",,,1
4,Jan 27 2023 10:31AM,256074,10,Bio-PRF,,,20
5,Jan 27 2023 10:29AM,256073,10,Eywa Pharma Inc.,,,5
6,Jan 27 2023 9:56AM,256070,10,ISDIN Corporation,,,1
7,Jan 27 2023 9:48AM,256067,10,"Senseonics, Incorporated",,,1
8,Jan 27 2023 9:44AM,256064,10,Bio-PRF,,,1
9,Jan 27 2023 9:40AM,256063,10,"Citieffe, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 10:58AM,256079,19,Opus Life Sciences,1,,
1,Jan 27 2023 10:57AM,256078,19,"Methapharm, Inc.",3,,
2,Jan 27 2023 10:57AM,256078,19,Methapharm-G,3,,
3,Jan 27 2023 10:51AM,256076,21,"NBTY Global, Inc.",1,,
4,Jan 27 2023 10:31AM,256074,10,Bio-PRF,20,,
5,Jan 27 2023 10:29AM,256073,10,Eywa Pharma Inc.,5,,
6,Jan 27 2023 9:56AM,256070,10,ISDIN Corporation,1,,
7,Jan 27 2023 9:48AM,256067,10,"Senseonics, Incorporated",1,,
8,Jan 27 2023 9:44AM,256064,10,Bio-PRF,1,,
9,Jan 27 2023 9:40AM,256063,10,"Citieffe, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 10:58AM,256079,19,Opus Life Sciences,1,,
1,Jan 27 2023 10:57AM,256078,19,"Methapharm, Inc.",3,,
2,Jan 27 2023 10:57AM,256078,19,Methapharm-G,3,,
3,Jan 27 2023 10:51AM,256076,21,"NBTY Global, Inc.",1,,
4,Jan 27 2023 10:31AM,256074,10,Bio-PRF,20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 10:58AM,256079,19,Opus Life Sciences,1.0,NaN,NaN
1,Jan 27 2023 10:57AM,256078,19,"Methapharm, Inc.",3.0,NaN,NaN
2,Jan 27 2023 10:57AM,256078,19,Methapharm-G,3.0,NaN,NaN
3,Jan 27 2023 10:51AM,256076,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jan 27 2023 10:31AM,256074,10,Bio-PRF,20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 10:58AM,256079,19,Opus Life Sciences,1.0,0.0,0.0
1,Jan 27 2023 10:57AM,256078,19,"Methapharm, Inc.",3.0,0.0,0.0
2,Jan 27 2023 10:57AM,256078,19,Methapharm-G,3.0,0.0,0.0
3,Jan 27 2023 10:51AM,256076,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jan 27 2023 10:31AM,256074,10,Bio-PRF,20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2560640,92.0,18.0,0.0
12,256038,1.0,0.0,0.0
16,768108,3.0,0.0,0.0
19,1024281,15.0,31.0,3.0
21,256076,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2560640,92.0,18.0,0.0
1,12,256038,1.0,0.0,0.0
2,16,768108,3.0,0.0,0.0
3,19,1024281,15.0,31.0,3.0
4,21,256076,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,92.0,18.0,0.0
1,12,1.0,0.0,0.0
2,16,3.0,0.0,0.0
3,19,15.0,31.0,3.0
4,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,92.0
1,12,Released,1.0
2,16,Released,3.0
3,19,Released,15.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,21
Status,,,,,
Completed,0.0,0.0,0.0,3.0,0.0
Executing,18.0,0.0,0.0,31.0,0.0
Released,92.0,1.0,3.0,15.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,21
0,Completed,0.0,0.0,0.0,3.0,0.0
1,Executing,18.0,0.0,0.0,31.0,0.0
2,Released,92.0,1.0,3.0,15.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,21
0,Completed,0.0,0.0,0.0,3.0,0.0
1,Executing,18.0,0.0,0.0,31.0,0.0
2,Released,92.0,1.0,3.0,15.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()